In [1]:
import pandas as pd
import numpy as np
# fix random seed for reproducibility
np.random.seed(7)

In [2]:
cleaned = pd.read_csv('datasets/cleaned_Izends_Data_Thru201712_ver5.csv',index_col=0)

/home/ambuj/izenda_fullcycle/environment/env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
cleaned_loss_des = pd.read_csv('hashed_cleaned_loss_desc_from_5_ipynb.csv',index_col=0)
# changing dtype of hashed_loss_desc_padded column
cleaned_loss_des['hashed_loss_desc_padded'] = cleaned_loss_des['hashed_loss_desc_padded'].apply(lambda x: np.fromstring(x[1:-1],dtype='int32',sep=' '))

In [4]:
cleaned_combined = pd.merge(cleaned,cleaned_loss_des[['EstimateTotal_bins','hashed_loss_desc_padded']],left_index=True,right_index=True)

In [5]:
cleaned_combined.shape

(766408, 28)

In [6]:
cleaned_combined.columns

Index(['DisplayName', 'DivisionName', 'City', 'Zip', 'State', 'LossYearMo',
       'StartedFlag', 'CommOrRes', 'NoteCount', 'PhotoCount', 'JobCount',
       'ClaimCount', 'EstimateTotal', 'PolicyHolderType', 'LossDescription',
       'Estimate_NetClaim', 'Estimate_MaterialSaleTax', 'Estimate_OverHead',
       'Estimate_Profit', 'Estimate_PctOverhead', 'Estimate_PctProfit',
       'Estimate_Deductible', 'Estimate_BaseSvcCharge',
       'CleanAddressFranchisorID', 'CleanAddressContactID', 'LossMo',
       'EstimateTotal_bins', 'hashed_loss_desc_padded'],
      dtype='object')

# Defining X and Y

In [7]:
X1 = cleaned_combined[['DisplayName', 'DivisionName','LossMo','CommOrRes','PolicyHolderType','NoteCount','PhotoCount','JobCount','ClaimCount','LossYearMo']]
X2 = cleaned_combined['hashed_loss_desc_padded']
y = cleaned_combined['EstimateTotal_bins']

# Getting Dummies

In [8]:
X1_dummies = pd.get_dummies(X1,sparse=True)

In [9]:
all_data = pd.merge(X1_dummies,pd.DataFrame(X2),left_index=True,right_index=True)
all_data['EstimateTotal_bins'] = y

# Splitting data

In [10]:
test2017=all_data[all_data["LossYearMo"].apply(lambda all_data:all_data>=201701)]
learning=all_data[all_data["LossYearMo"].apply(lambda all_data:all_data<201701)]

In [11]:
learning_x = learning[learning.columns.difference(['LossYearMo','EstimateTotal_bins'])]
learning_y = learning['EstimateTotal_bins']
learning_y_dummies = pd.get_dummies(learning_y,sparse=True)

In [12]:
test2017_x1 = test2017[test2017.columns.difference(['LossYearMo','EstimateTotal_bins','hashed_loss_desc_padded'])]
test2017_x2 = test2017['hashed_loss_desc_padded']
test2017_y = test2017['EstimateTotal_bins']
test2017_y_dummies = pd.get_dummies(test2017_y,sparse=True)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(learning_x, learning_y_dummies, test_size=0.33, random_state=42)

In [14]:
X1_train = X_train[X_train.columns.difference(['hashed_loss_desc_padded'])]
X2_train = X_train['hashed_loss_desc_padded']

In [15]:
X1_val = X_val[X_val.columns.difference(['hashed_loss_desc_padded'])]
X2_val = X_val['hashed_loss_desc_padded']

# Building Custom Keras Combined Model using Functional API

In [16]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint, History 
from keras.layers.merge import concatenate

Using TensorFlow backend.


In [17]:
# first input
input1 = Input(shape=(X1_train.shape[1],),name='input1')

In [18]:
# second input
input2 = Input(shape=(14,),name='input2')
embedding = Embedding(11024,100,input_length=14)(input2)
lstm = LSTM(15)(embedding)

In [19]:
# aux output
auxiliary_output = Dense(y_train.shape[1], activation='softmax', name='aux_output')(lstm)

In [20]:
# merge layers
merge = concatenate([input1,lstm])

In [21]:
hidden1 = Dense(512, activation='relu')(merge)
dropout1 = Dropout(0.25)(hidden1)
hidden2 = Dense(512, activation='relu')(dropout1)
dropout2 = Dropout(0.25)(hidden2)

In [22]:
# output layer
output = Dense(y_train.shape[1], activation='softmax',name='output')(dropout2)

In [23]:
# Model
model = Model(inputs=[input1,input2], outputs=[auxiliary_output,output])

In [24]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input2 (InputLayer)             (None, 14)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 14, 100)      1102400     input2[0][0]                     
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 15)           6960        embedding_1[0][0]                
__________________________________________________________________________________________________
input1 (InputLayer)             (None, 1064)         0                                            
__________________________________________________________________________________________________
concatenat

In [25]:
plot_model(model,to_file='8_combined_model.png')

In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'],loss_weights=[1., 0.2])

In [26]:
early_stop = EarlyStopping(monitor='val_loss', patience=50,restore_best_weights=True)

In [ ]:
model.fit({'input1':X1_train,'input2':np.stack(X2_train, axis=0)},
          {'output':y_train.values, 'aux_output': y_train.values},
          validation_data=({'input1':X1_val,'input2':np.stack(X2_val, axis=0)},{'output':y_val.values,'aux_output':y_val.values}),
         callbacks=[early_stop],epochs=500, batch_size=128,verbose=1)

Train on 335176 samples, validate on 165087 samples
Epoch 1/500
335176/335176 [==============================] - 87s 258us/step - loss: 2.6360 - aux_output_loss: 2.2192 - output_loss: 2.0844 - aux_output_acc: 0.1658 - output_acc: 0.2093 - val_loss: 2.6056 - val_aux_output_loss: 2.2013 - val_output_loss: 2.0215 - val_aux_output_acc: 0.1732 - val_output_acc: 0.2289
Epoch 2/500
335176/335176 [==============================] - 85s 253us/step - loss: 2.5815 - aux_output_loss: 2.1800 - output_loss: 2.0077 - aux_output_acc: 0.1850 - output_acc: 0.2368 - val_loss: 2.5990 - val_aux_output_loss: 2.1990 - val_output_loss: 2.0002 - val_aux_output_acc: 0.1738 - val_output_acc: 0.2370
Epoch 3/500
335176/335176 [==============================] - 85s 254us/step - loss: 2.5506 - aux_output_loss: 2.1543 - output_loss: 1.9818 - aux_output_acc: 0.1979 - output_acc: 0.2456 - val_loss: 2.6089 - val_aux_output_loss: 2.2071 - val_output_loss: 2.0090 - val_aux_output_acc: 0.1741 - val_output_acc: 0.2349
Epoch 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



335176/335176 [==============================] - 85s 254us/step - loss: 2.2097 - aux_output_loss: 1.8720 - output_loss: 1.6886 - aux_output_acc: 0.3089 - output_acc: 0.3698 - val_loss: 3.0223 - val_aux_output_loss: 2.5560 - val_output_loss: 2.3316 - val_aux_output_acc: 0.1629 - val_output_acc: 0.2271
Epoch 21/500
335176/335176 [==============================] - 85s 253us/step - loss: 2.2000 - aux_output_loss: 1.8641 - output_loss: 1.6797 - aux_output_acc: 0.3122 - output_acc: 0.3734 - val_loss: 3.0284 - val_aux_output_loss: 2.5662 - val_output_loss: 2.3107 - val_aux_output_acc: 0.1614 - val_output_acc: 0.2274
Epoch 22/500
335176/335176 [==============================] - 85s 254us/step - loss: 2.1909 - aux_output_loss: 1.8565 - output_loss: 1.6719 - aux_output_acc: 0.3146 - output_acc: 0.3757 - val_loss: 3.0614 - val_aux_output_loss: 2.5895 - val_output_loss: 2.3592 - val_aux_output_acc: 0.1612 - val_output_acc: 0.2251
Epoch 23/500
335176/335176 [==============================] - 85s 25

# Calculating Test Accuracy

In [29]:
test2017_preds_proba = model.predict({'input1':test2017_x1,'input2':np.stack(test2017_x2, axis=0)})

In [35]:
test2017_preds = np.argmax(test2017_preds_proba[0],axis=1)

In [37]:
mapping_dict = {0:'a',1:'b',2:'c',3:'d',4:'e',5:'f',6:'g',7:'h',8:'i',9:'j'}

In [38]:
test2017_preds_names = pd.Series(test2017_preds).map(mapping_dict)

In [39]:
from sklearn.metrics import accuracy_score

In [40]:
accuracy_score(test2017_y,test2017_preds_names)

0.1621785117135396